In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 34.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

#Read data from chunks_df.csv
df = pd.read_csv('/content/drive/MyDrive/INLPT_Project/csv_files/chunks_df.csv')

In [ ]:
# Display the DataFrame with embeddings
df

,Unnamed: 0,text,section,article,number1,number2,number3,number4,number5,number6,CELEX number,embedding
0,0,Avis juridique important,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[ -0.71743995 -2.661333 -0.5654966 2.04...
1,1,Special Report No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-3.5942285e+00 -1.9066961e+00 1.9812379e+00 ...
2,2,SPECIAL REPORT No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-2.76330781e+00 -3.82428586e-01 -1.05377293e+...
3,3,SPECIAL REPORT No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-3.6374424 -2.093941 2.3462052 0.805499...
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
...,...,...,...,...,...,...,...,...,...,...,...,...
29602,29602,For the High Authority,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-3.7505999 -2.234225 0.9126675 2.350292...
29603,29603,The President,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-2.93085 -0.78436 2.562405 2.02705 ...
29604,29604,Dino DEL BO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-2.2211666 -2.5205266 2.42058 -2.599566...
29605,29605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [ ]:
import numpy as np
import faiss

# Find the maximum length of embeddings
max_length = df['embedding'].apply(len).max()
print(max_length)

# Pad or truncate embeddings to a fixed size
df['embedding'] = df['embedding'].apply(lambda emb: np.pad(emb, (0, max_length - len(emb)), 'constant'))

# Convert the list of embeddings to a 2D numpy array
embeddings_array = np.vstack(df['embedding'].to_numpy())

# Check if there are valid embeddings
if embeddings_array.shape[0] == 0:
    print("No valid embeddings found.")
else:
    # Print the embeddings
    print(embeddings_array)
    print(embeddings_array.shape)

    # Initialize Faiss index
    index = faiss.IndexFlatIP(embeddings_array.shape[1])


    # Add embeddings to the index
    index.add(embeddings_array)


# Save Faiss index and embeddings_array in the drive
faiss.write_index(index, '/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_index.index')
np.save('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_embeddings.npy', embeddings_array)


300
[[-0.71743995 -2.661333   -0.5654966  ...  2.4868302  -1.3577734
  -2.7677662 ]
 [-3.5942285  -1.9066961   1.9812379  ... -3.4405668   0.4281069
   1.1571586 ]
 [-2.7633078  -0.3824286  -1.0537729  ... -2.8274567  -1.6278757
   1.1858498 ]
 ...
 [-2.2211666  -2.5205266   2.42058    ...  0.23477332  1.6267476
  -1.1239    ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
(29607, 300)


In [ ]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Provide a query text
query = "On what date does the Agreement on Partnership and Cooperation was signed by the European Communities and their Members States and Kazakhstan?"
query_embedding = nlp(query).vector


In [ ]:
import numpy as np
import faiss
import spacy
from sklearn.metrics.pairwise import cosine_similarity

# Load Spacy model
nlp = spacy.load("en_core_web_md")

# Load Faiss index
index = faiss.read_index('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_index.index')

# Load embeddings array
embeddings_array = np.load('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_embeddings.npy')

"""
def similarity_search(query_embedding, index, embeddings_array, k=5):
    # Find the k most similar embeddings to the query using Faiss
    _, indices = index.search(np.array([query_embedding]), k)

    # Extract the corresponding text from the DataFrame
    similar_texts = df.iloc[indices[0]]['text'].tolist()

    return similar_texts
"""

# Calculate cosine similarity between the query and all embeddings in the array
similarity_scores = cosine_similarity([query_embedding], embeddings_array)[0]


# Get the indices of the sorted similarity scores
sorted_indices = np.argsort(similarity_scores)

# Get the indices of scores less than 0.1
min_similarity_score_indices = [index for index in sorted_indices if (similarity_scores[index] < 0.1) and (similarity_scores[index] != 0.0)]

#print(min_similarity_score_indices)


#Similarity score sorted in asending order
#similarity_score_sorted = sorted(similarity_scores)
#print(min_similarity_score_sorted)

#min_similarity_score = [score for score in similarity_score_sorted if score < 0.1]
#print(similarity_score_array)

#min_similarity_index = np.argmin(similarity_scores)

#print(min_similarity_index)

#print(embeddings_array[min_similarity_index])

#print(f"   Text: {df['text'].iloc[min_similarity_index]}")


#top = np.where((similarity_scores < 0.1) & (similarity_scores != 0.0))[0]

# Get the indices of similarity scores less than 0.1 but not equal to 0.0
#top_matches_indices = np.where((similarity_scores < 0.1) & (similarity_scores != 0.0))[0]

# Get the top 10 indices in ascending order of similarity scores
#top_10_indices = np.argsort(similarity_scores[top_matches_indices])[:1]


In [ ]:
# Print the results
print("Top Similar Embeddings:")
for i, index in enumerate(min_similarity_score_indices, start=1):
    print(f"{i}. Similarity Score: {similarity_scores[index]}")
    print(embeddings_array[index])
    print()

Streaming output truncated to the last 5000 lines.
 -2.685716    1.071648   -0.7324381  -0.7892893   3.1213698  -0.27857602
  1.17248    -2.993024   -0.38190395  0.09390797  1.578788    1.98999
  2.30374    -1.190176   -0.5624968  -0.4047566  -0.7763759  -1.9247401
 -0.47684604  1.873238    1.3348001  -0.501276    0.584308   -1.30607   ]

622. Similarity Score: 0.009298160672187805
[ 0.59717596 -3.21338     4.096182   -1.9249799  -1.1467779   3.242976
  2.0035598  -0.7985281  -1.839916   -0.15253606 -0.243452   -1.315706
 -0.3047201  -2.49606     0.82695997 -0.5093724   0.26019207  1.7377322
 -1.0594599   0.56157196 -1.603558   -3.40606    -1.5909278   3.4048202
  0.38851994  1.3937919   0.792504    4.8880196   0.7611559   0.839174
 -1.2543961   0.76163596 -1.2585716   1.0232279  -0.07042401 -1.1567883
  1.6204319  -0.17559    -2.0999038   1.3913441   2.648356    0.17524405
 -0.897116    2.08254    -0.91387606 -0.84702796  0.02222404 -1.6149563
 -1.413412   -4.60872     2.705652   -2.6

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
 -5.4431434   1.6019665  -1.7748255  -0.33593217 -1.3245703  -3.8054085
  0.98933345 -3.1785972   0.4373393  -5.1072884  -2.4053798  -2.9922006
 -2.3750346  -1.9229715   0.45794994  3.3188088   1.8440988  -3.0310092
  4.3521733   3.6199553  -1.8106464   1.2353188  -0.01304708 -0.14856926
 -2.1521058   3.9244506   2.6773357  -4.690563    2.5666995  -4.678506
  1.0399427   0.2100287   3.3487904  -3.4713159   2.7860637  -0.5945143
 -2.2123165  -2.8309722   0.30813646 -0.1111915   4.1687236   2.9736645
  2.217752   -2.4077086   4.4658704  -1.9600372  -0.7157815   5.26473
  5.9898076   2.7089593  -1.5480287   0.3504214   1.7167699   2.9106088
 -2.6566665   1.7525779  -2.7272758  -0.17550918 -5.52968     1.0611428
  0.45408127  4.896548   -0.02389286  1.9519999  -2.4056356  -0.08466385
 -1.6098621  -1.0176073   0.11528647 -5.5618935  -3.0911796   1.0752164
  0.5993129   5.416886   -1.8506057  -0.8802151   2.2297027  -0.48243722
 -3.127541   

In [ ]:
# Print the text for the top similar embeddings
print("Top Similar Texts:")
seen_similarity_scores = set()

for i, index in enumerate(min_similarity_score_indices, start=1):
    similarity_score = similarity_scores[index]

    # Check if this similarity score has been seen before
    if similarity_score not in seen_similarity_scores:
        seen_similarity_scores.add(similarity_score)

        print(f"{i}. Similarity Score: {similarity_score}")
        print(f"   Text: {df['text'].iloc[index]}")
        print()


Top Similar Texts:
1. Similarity Score: -0.1738460510969162
   Text: *****

7. Similarity Score: -0.14677216112613678
   Text: 2.Uranium-235 | Unirradiated [2] | | | |

8. Similarity Score: -0.13752099871635437
   Text: Sirs:

11. Similarity Score: -0.13036580383777618
   Text: Mr/Ms …

12. Similarity Score: -0.11976351588964462
   Text: HAVE ADOPTED THIS DIRECTIVE:

16. Similarity Score: -0.1131666824221611
   Text: TITLE I

20. Similarity Score: -0.1127748042345047
   Text: Date: …………

29. Similarity Score: -0.108711376786232
   Text: HAVE ADOPTED THIS DECISION:

30. Similarity Score: -0.10845077037811279
   Text: SECTION I

31. Similarity Score: -0.10461454093456268
   Text: I

45. Similarity Score: -0.10005579888820648
   Text: SF=(FBAT*0,30)+ (FDA*0,30)+ (FBC*0,10)+(FFFC*0,05)+(FRFC *0,05)+(FEC *0,05)+(FBUT*0,05)+(FMIC*0,05)+(FSPK*0,05)

47. Similarity Score: -0.09894643723964691
   Text: HAVE ADOPTED THIS REGULATION:

50. Similarity Score: -0.09765318036079407
   Text: COUNTRY: H

In [ ]:

for index in enumerate(min_similarity_score_indices, start=1):
    #similarity_score = similarity_scores[index]
print(seen_similarity_scores[index])

TypeError: 'set' object is not subscriptable

In [ ]:
'''
import numpy as np
import faiss
import spacy

# Load Spacy model
nlp = spacy.load("en_core_web_md")

# Load Faiss index
index = faiss.read_index('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_index.index')

# Load embeddings array
embeddings_array = np.load('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_embeddings.npy')

query_embedding1 = nlp("the Agreement on Partnership and Cooperation was signed by the European Communities and their Members States and Kazakhstan").vector

# Add a batch dimension to the query embedding
query_embedding1 = np.expand_dims(query_embedding1, axis=0)

# Perform the search
D, I = index.search(query_embedding1, k=200)
'''


In [ ]:
I

array([[27815, 27806, 21342, 21313, 14268, 13855, 13625, 13466, 13461,
        10696,  9762,  2166,  2113,   892, 28033,   430,    45,  5327,
        25631,  5388]])

In [ ]:
# Extract the corresponding text from the DataFrame
#similar_texts = df.iloc[I[0]]['text'].tolist()

#print(similar_texts)

['and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'and', 'Implementation of the Agreement', 'Management of the operations', 'Management of the operations', 'Assessment by the Commission', 'of the other part,', 'of the other part,', 'of the other part,', 'Transparency of the operations', 'Transparency of the operations', 'Modalities of cooperation', 'the protection of the environment;', 'Standardisation of communication', 'Member of the Commission of the European Communities', 'Member of the Commission of the European Communities', 'Member of the Commission of the European Communities', 'Member of the Commission of the European Communities', 'Member of the Commission of the European Communities', 'the insensitivity of the regulation; and', 'Participation of the recipients', 'the accuracy of the regulation; and', 'the accuracy of the regulation; and', 'President of the Commission', 'Responsibilities of the importer', 'Member of the Commission', '

In [ ]:
#print("Query Embedding Dimension:", query_embedding1.shape)
#print("Index Embedding Dimension:", embeddings_array.shape[1])


Query Embedding Dimension: (1, 300)
Index Embedding Dimension: 300


In [ ]:
%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install transformers
%pip install sentence-transformers
%pip install datasets
%pip install accelerate
%pip install einops
%pip install langchain
#%pip install xformers
%pip install bitsandbytes
%pip install matplotlib seaborn tqdm
%pip install evaluate
%pip install rouge_score
%pip install bert_score

In [ ]:
%pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
from transformers import LlamaTokenizer
hf_auth = os.environ.get('HF_AUTH')
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf",use_auth_token=hf_auth)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
from torch import cuda, bfloat16
import os
import transformers
model_id = 'meta-llama/Llama-2-13b-chat-hf'

In [ ]:
bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


In [ ]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=os.environ.get('HF_AUTH')
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
#Loading the model will take some time, (roughly 5 min)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bitsAndBites_config,
    device_map='auto',
    token=os.environ.get('HF_AUTH')
)

model.eval()# we only use the model for inference
print(f"Model loaded ")

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
model.get_memory_footprint()

7134302208

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    # we pass model parameters here too
    temperature=0.01,
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
### your code ###
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)
### your code ###

In [ ]:
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS

# Create a FaissIndexStore
vectorstore = FAISS(query_embedding, index, df['text'],1)


In [ ]:
seen_similarity_scores = set()

# Assuming you have similarity_scores and top_matches_indices
for i, index in enumerate(min_similarity_score_indices, start=1):
    similarity_score = similarity_scores[index]

    # Check if this similarity score has been seen before
    if similarity_score not in seen_similarity_scores:
        seen_similarity_scores.add(similarity_score)

        related_text = df['text'].iloc[index]

        # Generate text based on the related text
        generated_text = generate_text(related_text)

        # Print the results
        print(f"{i}. Similarity Score: {similarity_score}")
        print(f"   Original Text: {related_text}")
        print(f"   Generated Text: {generated_text}")
        print()
